In [1]:
pip install spotipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# making http requests
import requests
#encoding and decoding data into base64 format
import base64


# for accessing Spotify API
import spotipy
# handling Spotify's OAuth Authentication
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

In [3]:
# Spotify Developer Credentials
CLIENT_ID = 'ef8951cc5a7743d9a8fd30bd1ba99288'
CLIENT_SECRET = 'c4745276f3c34894832c9935e54dd60e'

# Encoding CLIENT_ID and CLIENT_SECRET into base64 format
client_credentials = f'{CLIENT_ID}:{CLIENT_SECRET}'
client_credentials_base64 = base64.b64encode(client_credentials.encode())

#Request the access token
token_url = 'https://accounts.spotify.com/api/token'

headers = {
    'Authorization' : f'Basic {client_credentials_base64.decode()}'
}

data = {
    'grant_type': 'client_credentials'
}

response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print(access_token)
    print('Access token obtained successfully')
else:
    print('Error Obtaining Access token')
    exit()

BQCB5HESZfFZJr1OFLCUFtBG-IZ9nfKz7bnxA-7mNyW5G9hY7tQrvPDjDd2uytRN1NypFP839tKjKrhs0LIbAgvL4dq1kuc5nGBF0_ZEPq8H45R-Iww
Access token obtained successfully


In [4]:
# Playlist id of spotify "big on the internet" playlist from trending 
playlist_id = '37i9dQZF1DXbVhgADFy3im'

In [5]:
def get_trending_playlist_data(playlist_id, access_token):
    
    #setup Spotify with the access token
    sp = spotipy.Spotify(auth=access_token)
    
    # get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id,name)))')
    
    # Extracting relevant features and storing in dictionaries
    music_data = []
    
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        
        # get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not Available' else None
        
        # get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not Available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None
        
        # get the popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not Available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None
        
        # Additional track information
        
        track_data = {
            'Track_name' : track_name,
            'Artists' : artists,
            'Album_Name' : album_name,
            'Album_id' : album_id,
            'Track_id' : track_id,
            'Popularity' : popularity,
            "Release Date" : release_date,
            'Duration' : audio_features['duration_ms'] if audio_features else None,
            'Explicit' : track_info.get('explicit', None),
            'External_urls' : track_info.get('external_urls', {}).get('spotify', None),
            'Danceability' : audio_features['danceability'] if audio_features else None,
            'Energy' : audio_features['energy'] if audio_features else None,
            'Key' : audio_features['key'] if audio_features else None,
            'Loudness' : audio_features['loudness'] if audio_features else None,
            'Mode' : audio_features['mode'] if audio_features else None,
            'Speechiness' : audio_features['speechiness'] if audio_features else None,
            'Acousticness' : audio_features['acousticness'] if audio_features else None,
            'Instrumentalness' : audio_features['instrumentalness'] if audio_features else None,
            'Liveness' : audio_features['liveness'] if audio_features else None,
            'Valence' : audio_features['valence'] if audio_features else None,
            'Tempo' : audio_features['tempo'] if audio_features else None,
        }
            
    
        music_data.append(track_data)
    
    
    df = pd.DataFrame(music_data)
    
    return df

In [6]:
music_df = get_trending_playlist_data(playlist_id, access_token)

In [7]:
music_df

,Track_name,Artists,Album_Name,Album_id,Track_id,Popularity,Release Date,Duration,Explicit,External_urls,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Pehle Bhi Main,"Vishal Mishra, Raj Shekhar",ANIMAL,0a183xiCHiC1GQd8ou7WXO,7yDHHVKLbvDmVw1XXhDDIO,91,2023-11-24,250158,False,https://open.spotify.com/track/7yDHHVKLbvDmVw1...,...,0.406,10,-10.230,0,0.0353,0.7680,0.000002,0.1070,0.281,76.944
1,Tu hai kahan,AUR,Tu hai kahan,73vfz6Szer7i3GFIesix6l,3zSSCPpLZ5Oc8nelhhGjKz,88,2023-10-16,263185,False,https://open.spotify.com/track/3zSSCPpLZ5Oc8ne...,...,0.356,1,-9.738,1,0.0750,0.4370,0.000065,0.1120,0.569,83.679
2,Apna Bana Le,"Sachin-Jigar, Arijit Singh, Amitabh Bhattacharya",Bhediya (Original Motion Picture Soundtrack),12sC6UjMWz6EaxnzyfCNMe,5bQ6oDLqvw8tywmnSmwEyL,81,2022-11-22,261702,False,https://open.spotify.com/track/5bQ6oDLqvw8tywm...,...,0.562,9,-7.399,1,0.0267,0.8030,0.000006,0.0630,0.436,93.982
3,"Satranga (From ""ANIMAL"")","Arijit Singh, Shreyas Puranik, Siddharth - Garima","Satranga (From ""ANIMAL"")",5mZX4EMwEyohNmVfLTDtXn,3yHyiUDJdz02FZ6jfUbsmY,89,2023-10-27,271169,False,https://open.spotify.com/track/3yHyiUDJdz02FZ6...,...,0.405,0,-5.810,1,0.0298,0.7350,0.000000,0.1890,0.534,89.480
4,One Love,Shubh,One Love,3tNH9Fp4dm3qa3Kr4cv5uJ,5ZLkihi6DVsHwDL3B8ym1t,86,2023-08-18,159222,False,https://open.spotify.com/track/5ZLkihi6DVsHwDL...,...,0.512,7,-7.757,1,0.1180,0.2840,0.006450,0.1010,0.881,144.926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Agar Tum Saath Ho,"Alka Yagnik, Arijit Singh",Tamasha,2CUXo26JAWIbQx0EVMnjpA,3hkC9EHFZNQPXrtl8WPHnX,78,2015-10-16,341055,False,https://open.spotify.com/track/3hkC9EHFZNQPXrt...,...,0.519,3,-8.744,1,0.0326,0.5570,0.000268,0.1720,0.415,122.925
71,Desi Kalakaar,Yo Yo Honey Singh,Desi Kalakaar,3IbKcMtNRze5t0ANwdp45H,7m4CgL81VVIHaaiSxyH922,78,2014-09-26,253353,False,https://open.spotify.com/track/7m4CgL81VVIHaai...,...,0.670,0,-4.587,1,0.2670,0.0796,0.000180,0.0908,0.766,156.062
72,Jhoome Jo Pathaan,"Vishal-Shekhar, Arijit Singh, Sukriti Kakar, V...",Pathaan,2ooWY7xriwwyj23dAI32C2,6FAYpZ4jve8vpvTwUvjK6H,78,2022-12-22,208164,False,https://open.spotify.com/track/6FAYpZ4jve8vpvT...,...,0.738,7,-7.639,1,0.0748,0.0964,0.000000,0.3310,0.616,104.964
73,Daku,"Chani Nattan, Inderpal Moga",Daku,3RaIXDlIRvziryGLXm8lBR,3wz2vWhYnnBoFmgMkqtzLy,77,2021-01-16,131078,False,https://open.spotify.com/track/3wz2vWhYnnBoFmg...,...,0.558,8,-5.855,0,0.0663,0.3250,0.000000,0.1350,0.495,98.062


In [8]:
music_df.isnull().sum()

Track_name          0
Artists             0
Album_Name          0
Album_id            0
Track_id            0
Popularity          0
Release Date        0
Duration            0
Explicit            0
External_urls       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [9]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track_name        75 non-null     object 
 1   Artists           75 non-null     object 
 2   Album_Name        75 non-null     object 
 3   Album_id          75 non-null     object 
 4   Track_id          75 non-null     object 
 5   Popularity        75 non-null     int64  
 6   Release Date      75 non-null     object 
 7   Duration          75 non-null     int64  
 8   Explicit          75 non-null     bool   
 9   External_urls     75 non-null     object 
 10  Danceability      75 non-null     float64
 11  Energy            75 non-null     float64
 12  Key               75 non-null     int64  
 13  Loudness          75 non-null     float64
 14  Mode              75 non-null     int64  
 15  Speechiness       75 non-null     float64
 16  Acousticness      75 non-null     float64
 17 

In [10]:
data = music_df

In [11]:
# function to calculate weighted popularity based on released date
def calculated_weighted_popuarity(release_date):
    
    #convert the release date into datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    
    #calculating the time span [release date and today]
    time_span = datetime.now() - release_date
    
    weight = 1 / (time_span.days + 1)
    
    return weight

In [12]:
calculated_weighted_popuarity('2023-11-24')

0.020833333333333332

In [13]:
data.sample(6)

,Track_name,Artists,Album_Name,Album_id,Track_id,Popularity,Release Date,Duration,Explicit,External_urls,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
66,Zihaal e Miskin,"Javed-Mohsin, Vishal Mishra, Shreya Ghoshal",Zihaal e Miskin,54ubCIO3voLnU7ZmJhxqBd,51DEaelXeJJ6cFFYbX8Hal,78,2023-05-25,243953,False,https://open.spotify.com/track/51DEaelXeJJ6cFF...,...,0.632,9,-7.166,0,0.0331,0.2590,0.00000,0.0903,0.386,128.844
16,Night Changes,One Direction,FOUR (Deluxe),4gCNyS7pidfK3rKWhB3JOY,5O2P9iiztwhomNh8xkR9lJ,89,2014-11-17,226600,False,https://open.spotify.com/track/5O2P9iiztwhomNh...,...,0.520,8,-7.747,1,0.0353,0.8590,0.00000,0.1150,0.370,120.001
46,Malang Sajna,"Sachet Tandon, Parampara Tandon, Kumaar, Sache...",Malang Sajna,7khRLZe8P6XcUoRgOa3nnf,73K33p4Vyz9koXGqmL5eFs,82,2022-12-19,160194,False,https://open.spotify.com/track/73K33p4Vyz9koXG...,...,0.716,10,-4.964,1,0.0370,0.2320,0.00000,0.3300,0.871,113.943
69,"Naa Ready (From ""Leo"")","Anirudh Ravichander, Thalapathy Vijay, Asal Ko...","Naa Ready (From ""Leo"")",7qRoSVSyHeA9qtOXUpBwRI,1B02UI29t3PTh3m98absaP,79,2023-06-22,248182,False,https://open.spotify.com/track/1B02UI29t3PTh3m...,...,0.805,1,-5.576,1,0.1530,0.0810,0.00784,0.0318,0.572,87.956
68,Excuses,"AP Dhillon, Gurinder Gill, Intense",Excuses,3GkXRRRkV3rfgwG1wJset9,29m79w9xPMH4YCD6r8JSmV,78,2020-07-24,176233,False,https://open.spotify.com/track/29m79w9xPMH4YCD...,...,0.717,5,-7.749,0,0.0822,0.0842,0.00000,0.1490,0.494,94.984
50,Still Rollin,Shubh,Still Rollin,5AivaZj0CiQJoDWqVH2pbh,6oz1L5jcqZWYuId7nxvIan,81,2023-05-19,174730,False,https://open.spotify.com/track/6oz1L5jcqZWYuId...,...,0.431,6,-11.093,0,0.0914,0.1820,0.00180,0.1130,0.246,94.000


In [14]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track_name        75 non-null     object 
 1   Artists           75 non-null     object 
 2   Album_Name        75 non-null     object 
 3   Album_id          75 non-null     object 
 4   Track_id          75 non-null     object 
 5   Popularity        75 non-null     int64  
 6   Release Date      75 non-null     object 
 7   Duration          75 non-null     int64  
 8   Explicit          75 non-null     bool   
 9   External_urls     75 non-null     object 
 10  Danceability      75 non-null     float64
 11  Energy            75 non-null     float64
 12  Key               75 non-null     int64  
 13  Loudness          75 non-null     float64
 14  Mode              75 non-null     int64  
 15  Speechiness       75 non-null     float64
 16  Acousticness      75 non-null     float64
 17 

In [15]:
# selecting columns with dtype int and float for Feature scaling
selected_columns = music_df.columns[music_df.dtypes.astype(str).isin(['int64', 'float64'])]
selected_columns

Index(['Popularity', 'Duration', 'Danceability', 'Energy', 'Key', 'Loudness',
       'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo'],
      dtype='object')

In [16]:
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 'Loudness','Mode', 'Speechiness',
                           'Acousticness', 'Instrumentalness', 'Liveness','Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [23]:
# content based recommendations using music features

def content_based_recommendation(input_song, num_recommendations = 6):
    
    # checking if song name is available in the playlist
    if input_song not in music_df['Track_name'].values:
        print(f"{input_song} is not found in dataset, Enter a valid song name")
        
        return 
    
    # get the index of input song in music_df
    input_song_index = music_df[music_df['Track_name'] == input_song].index[0]
    
    # calculating the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)
    
    # similar song indices
    similar_song_indices = similarity_scores.argsort()[0][::-1][1: num_recommendations+1]
    
    # names of similar songs based on content-based filtering
    content_based_recom = music_df.iloc[similar_song_indices][['Track_name', 'Artists','Album_Name', 'Release Date', 'Popularity']]
    
    return content_based_recom
    

In [24]:
content_based_recommendation('Naa Ready (From "Leo")')

,Track_name,Artists,Album_Name,Release Date,Popularity
40,Chaleya,"Anirudh Ravichander, Arijit Singh, Shilpa Rao,...",Jawan,2023-09-05,83
28,No Lie,"Sean Paul, Dua Lipa",Mad Love The Prequel,2018-06-29,84
71,Desi Kalakaar,Yo Yo Honey Singh,Desi Kalakaar,2014-09-26,78
13,Dandelions,Ruth B.,Safe Haven,2017-05-05,90
6,With You,AP Dhillon,With You,2023-08-11,81
48,love nwantiti (ah ah ah),CKay,CKay The First,2019-08-30,81


In [36]:
# hybrid recommendation based on content and popularity

def hybrid_recommendations(input_song_name, num_recommendations = 8, alpha = 0.5):
    if input_song_name not in music_df['Track_name'].values:
        print(f"{input_song_name} not found in dataset, Enter a valid song name")
        return 
    
    # Get content based recommendations
    content_based_rec = content_based_recommendation(input_song_name, num_recommendations)
    
    # Popularity of the song
    popularity_score = music_df.loc[music_df['Track_name'] == input_song_name, 'Popularity'].values[0]

    # weighted popularity score
    weighted_popularity_score = popularity_score * calculated_weighted_popuarity(music_df.loc[music_df['Track_name'] == input_song_name, 'Release Date'].values[0])
    
    # combining content-based and popularity based recommendations
    hybrid_recom = content_based_rec
    hybrid_recom = hybrid_recom.append({
        'Track_name' : input_song_name,
        "Artists" : music_df.loc[music_df['Track_name'] == input_song_name, 'Artists'].values[0],
        "Album_Name" : music_df.loc[music_df['Track_name'] == input_song_name, 'Album_Name'].values[0],
        'Release Date' : music_df.loc[music_df['Track_name'] == input_song_name, "Release Date"].values[0],
        "Popularity" : weighted_popularity_score,
    }, ignore_index = True)
    
    # sorting hybrid recommendations based on weighted popularity score
    hybrid_recom = hybrid_recom.sort_values(by = "Popularity", ascending = False)
    
    # Removing input song from recommendations
    hybrid_recom = hybrid_recom[hybrid_recom['Track_name'] != input_song_name]
    
    return hybrid_recom
    

In [37]:
hybrid_recommendations("Gangsta's Paradise")

C:\Users\User\AppData\Local\Temp\ipykernel_7608\2366038450.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recom = hybrid_recom.append({


,Track_name,Artists,Album_Name,Release Date,Popularity
7,Until I Found You (with Em Beihold) - Em Beiho...,"Stephen Sanchez, Em Beihold",Until I Found You (Em Beihold Version),2022-04-22,90.0
3,Unholy (feat. Kim Petras),"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),2022-09-22,85.0
5,No Lie,"Sean Paul, Dua Lipa",Mad Love The Prequel,2018-06-29,84.0
6,"Abrar’s Entry | Jamal Kudu (From ""ANIMAL"")","Harshavardhan Rameshwar, Choir","Abrar’s Entry | Jamal Kudu (From ""ANIMAL"")",2023-12-06,83.0
1,Malang Sajna,"Sachet Tandon, Parampara Tandon, Kumaar, Sache...",Malang Sajna,2022-12-19,82.0
4,love nwantiti (ah ah ah),CKay,CKay The First,2019-08-30,81.0
2,"Badass (From ""Leo"")",Anirudh Ravichander,"Badass (From ""Leo"")",2023-09-28,79.0
0,Jhoome Jo Pathaan,"Vishal-Shekhar, Arijit Singh, Sukriti Kakar, V...",Pathaan,2022-12-22,78.0
